In [1]:
from ibpythonic import ibConnection, message
from ibapi.contract import Contract

from time import sleep, strftime
from datetime import datetime

import pandas as pd
import numpy as np

In [2]:
def makeStkContract(contractTuple):
    newContract = Contract()
    newContract.symbol = contractTuple[0]
    newContract.secType = contractTuple[1]
    newContract.exchange = contractTuple[2]
    newContract.currency = contractTuple[3]
    return newContract

In [3]:
bar_data_list = []
busy = False
active_symbol = ''

In [4]:
def my_hist_data_handler(msg):
    global bar_data_list
    # print('Receiving bar: ' + datetime.now().strftime("%H:%M:%S.%f"))
    bar_data_list.append(msg.bar)

In [5]:
def my_hist_data_end_handler(msg):
    global bar_data_list
    global busy
    global active_symbol
        
    # print('Creating df: ' + datetime.now().strftime("%H:%M:%S.%f"))
    df = pd.DataFrame(columns=('date', 'open', 'high', 'low', 'close', 'volume'))
    for bar in bar_data_list:
        row = {'date': bar.date, 'open': bar.open, 'high': bar.high, 'low': bar.low, 'close': bar.close, 'volume': bar.volume}
        df = df.append(row, ignore_index=True)
    df.loc[:, 'date'] = pd.to_datetime(df.date, format='%Y%m%d') #'%Y%m%d  %H:%M:%S'
    df = df.set_index('date')

    # print('Writing CSV: ' + datetime.now().strftime("%H:%M:%S.%f"))
    df.to_csv('data/'+active_symbol+'.csv')
    
    bar_data_list = []
    active_symbol = ''
    busy = False

In [6]:
symbols = ['TSLA', 'ADBE', 'NVDA', 'AMD', 'INTL']
endtime = strftime('%Y%m%d %H:%M:%S')

In [7]:
con = ibConnection(port=7497)
con.register(my_hist_data_handler, message.historicalData)
con.register(my_hist_data_end_handler, message.historicalDataEnd)

True

In [8]:
con.connect()
    
for symbol in symbols:
    global busy

    # print('Creating contract: ' + datetime.now().strftime("%H:%M:%S.%f"))
    contractTuple = (symbol, 'STK', 'ISLAND', 'USD')
    stkContract = makeStkContract(contractTuple)

    active_symbol = stkContract.symbol
    # print('Requesting data: ' + datetime.now().strftime("%H:%M:%S.%f"))
    busy = True
    con.reqHistoricalData(1,stkContract,endtime,"10 Y","1 day","MIDPOINT",1,1, keepUpToDate=False, chartOptions=[])
    
    while busy is True:
        sleep(0.5)
    # print('Finished symbol: ' + datetime.now().strftime("%H:%M:%S.%f"))
    
    sleep(1)

con.disconnect()

Creating contract: 15:43:38
Requesting data: 15:43:38
Receiving bar: 15:43:41
Receiving bar: 15:43:41
Receiving bar: 15:43:41
Receiving bar: 15:43:41
Receiving bar: 15:43:41
Receiving bar: 15:43:41
Receiving bar: 15:43:41
Receiving bar: 15:43:41
Receiving bar: 15:43:41
Receiving bar: 15:43:41
Receiving bar: 15:43:41
Receiving bar: 15:43:41
Receiving bar: 15:43:41
Receiving bar: 15:43:41
Receiving bar: 15:43:41
Receiving bar: 15:43:41
Receiving bar: 15:43:41
Receiving bar: 15:43:41
Receiving bar: 15:43:41
Receiving bar: 15:43:41
Receiving bar: 15:43:41
Receiving bar: 15:43:41
Receiving bar: 15:43:41
Receiving bar: 15:43:41
Receiving bar: 15:43:41
Receiving bar: 15:43:41
Receiving bar: 15:43:41
Receiving bar: 15:43:41
Receiving bar: 15:43:41
Receiving bar: 15:43:41
Receiving bar: 15:43:41
Receiving bar: 15:43:41
Receiving bar: 15:43:41
Receiving bar: 15:43:41
Receiving bar: 15:43:41
Receiving bar: 15:43:41
Receiving bar: 15:43:41
Receiving bar: 15:43:41
Receiving bar: 15:43:41
Receiving 

True

unhandled exception in EReader thread
Traceback (most recent call last):
  File "c:\users\martin\dev\_pyvenvs\latest36\lib\site-packages\ibapi\reader.py", line 34, in run
    data = self.conn.recvMsg()
  File "c:\users\martin\dev\_pyvenvs\latest36\lib\site-packages\ibapi\connection.py", line 99, in recvMsg
    buf = self._recvAllMsg()
  File "c:\users\martin\dev\_pyvenvs\latest36\lib\site-packages\ibapi\connection.py", line 119, in _recvAllMsg
    buf = self.socket.recv(4096)
OSError: [WinError 10038] Ein Vorgang bezog sich auf ein Objekt, das kein Socket ist


In [19]:
 con.disconnect()

False

- dict of symbols with ascending numbers
- download data one after another, symbol-number is id for downloading
- write received data to file as fast as possible, filename is symbol-number
- download next symbol as soon as data is in file
- in parallel load finished data from files, convert to dataframes and save as csv

In [ ]:
with open('your_file.txt', 'w') as f:
    for item in my_list:
        f.write("%s\n" % item)

# or

outfile.write("\n".join(itemlist))

# or

import pickle

with open('outfile', 'wb') as fp:
    pickle.dump(itemlist, fp)
    
with open ('outfile', 'rb') as fp:
    itemlist = pickle.load(fp)
    
# or

open("bla.txt", "wb").write(''.join(random.choice(string.ascii_lowercase) for i in xrange(10**7)))

- check for existing data
- start time for downloading is end time of data. last candle of data is considered invalid
- check for splits/mergers

- if symbol is multiple times in list
    - load data from primary exchange if ppossible
    - if primary is not available, load from exchange with highest average (price-)volume


- pull symbols from ib webpage
- create symbol database
- pull symbol data from ib
- abillity to create own univestes (=symbol lists) based on symbol data

- store prica data in CSV per symbol per timframe
- store price data in database? sqlite, as handeled in one file?
- cloud storage of data